# Simulation for 1 batch of 20 nodes, starting month 1

In [1]:
from lib import *
from params_bom_hardware_components import *
startmonth = 24

## input arguments

In [2]:
from params_environment import *
md_code("params_environment.py")

```python


environment.cost_power_kwh = "0.15 USD"
environment.cost_rack_unit = "12 USD"
environment.bandwidth_mbit = 20

# sales arguments
environment.sales_price_cu = "10 USD"
environment.sales_price_su = "6 USD"

device_edge = simulation.device_get("edge1")
# print(d1)

switch = simulation.device_get("switch")

environment.device_add("edge1", device_edge, 20)
environment.device_add("switch", switch, 2)

# month:growth_percent of nodes being added
simulation.growth_percent_set("3:5,11:8,24:10,36:12,48:10,60:10,61:0")

# means at end of period we produce 40% more cpr (*1.4)
# cpr = capacity production rate (is like hashrate of bitcoin)
simulation.cpr_improve_set("71:40")

# price of a capacity unit goes down over time, here we say it will go down 40%
# means we expect price to be lowered by X e.g. 40 (*0.6)
simulation.cpr_sales_price_decline_set("0:0,71:40")

# utilization of the nodes, strating with 0
simulation.utilization_set("20:80,40:90")

# super important factor, how does token price goes up, this is ofcourse complete speculation, no-one knows
simulation.tokenprice_set("0:0.15,71:2")

# simulation.difficulty_level_set("0:2,71:50")
simulation.difficulty_level_set("0:2,71:2")

```

## Template Environment Overview (20 servers)

In [3]:
environment


## threefold.simulation.environment
 - name                : default
 - description         : 
 - cost_power_kwh      : 0.15
 - cost_rack_unit      : 12
 - cost                : 81,540
 - power               : 4,640
 - rackspace_u         : 402
 - cru                 : 320
 - sru                 : 20000
 - hru                 : 1200000
 - mru                 : 2560
 - su                  : 1166
 - cu                  : 634
 - cu_passmark         : 771
 - su_perc             : 62.95%
 - cu_perc             : 37.05%
 - cpr                 : 2,118
 - cost_su_capex       : 44.0
 - cost_cu_capex       : 47.6
 - cost_power          : 501
 - cost_rack           : 4,824
 - cost_month          : 6,684
 - cost_cu_month       : 3.9
 - cost_su_month       : 3.61
 - bandwidth_mbit      : 20
 - cost_bandwidth      : 0
 - nr_devices          : 20
 - sales_price_cu      : 10
 - sales_price_su      : 6
 - sales_price_nu      : 0
 - sales_price_total   : 13,345
 - sales_price_cpr_unit: 6.3


## Node Overview (= 1 server)

In [4]:
device_edge


## threefold.simulation.device
 - name                : edge1
 - description         : 
 - device_template_name: edge1
 - cost                : 3,627
 - power               : 222
 - rackspace_u         : 20
 - cru                 : 16
 - sru                 : 1000
 - hru                 : 60000
 - mru                 : 128
 - su                  : 58
 - cu                  : 31
 - cu_passmark         : 771
 - su_perc             : 62.95%
 - cu_perc             : 37.05%
 - cpr                 : 105
 - cost_su_capex       : 39.1
 - cost_cu_capex       : 42.3
 - cost_power          : 0
 - cost_rack           : 0
 - cost_month          : 60.5
 - cost_cu_month       : 0.71
 - cost_su_month       : 0.65


In [5]:
md(f"""
### Device Details

- nr of TB netto usable = {round(device_edge.su,1)} TB
- nr of cpu cores = {round(device_edge.cru,1)} Cores
- power used = {device_edge.power} watt
- performance in passmark = {round(device_edge.cu_passmark*device_edge.cu)}

""")


### Device Details

- nr of TB netto usable = 58.3 TB
- nr of cpu cores = 16.0 Cores
- power used = 222 watt
- performance in passmark = 24479



In [6]:
md(f"## Simulate Return of 20 Nodes starting at month {startmonth}")

## Simulate Return of 20 Nodes starting at month 24

In [7]:
nb=simulation.nodesbatch_calc(month=startmonth, nrnodes_new=20)
nb._data

## threefold.simulation.nodesbatch
 - batch_nr            : 24
 - nrnodes             : 20
 - month_start         : 24
 - months_left         : 60
 - tft_farmed_before_simulation: 0

    ## threefold.simulation.nodesbatch.node
     - rackspace_u         : 20
     - cost_hardware       : 4,077
     - cpr                 : 120
     - power               : 232
    



In [8]:
md(f"""
- Hardware Investment Cost for the node batch : USD {nb.cost_hardware}
- ROI In Months                 : {nb.roi_months}
""")


- Hardware Investment Cost for the node batch : USD 81,540
- ROI In Months                 : 9


In [9]:
plt.figure(title='TFT Farmed, Cultivated, Sold, Burned')
names=["farmed","cultivated","sold","burned","total"]
colors=["orange","green","blue","yellow","red"]
pos=[1,1,-1,-1,1]
x=30
for i in range(5):
    name=names[i]
    color=colors[i]
    cells=[]
    for x in range(startmonth,60+startmonth):
        y=nb.sheet.rows["tft_%s"%name].cells[x]*pos[i]
        cells.append(y)    
    line=plt.plot(np.arange(60),cells,colors=[color])
    label = plt.label([name], x=[30],y=[cells[30]+100], colors=[color])

plt.show()

In [10]:
plt.figure(title='Income in USD for the tokens')
names=["farmed","cultivated","sold","burned","total"]
colors=["orange","green","blue","yellow","red"]
pos=[1,1,-1,-1,1]
for i in range(5):
    name=names[i]
    color=colors[i]
    cells=[]
    for x in range(startmonth,60+startmonth):
        y=nb.sheet.rows["tft_%s"%name].cells[x]*float(simulation.tft_price_get(x))*pos[i]  
        cells.append(y)
    line=plt.plot(np.arange(60),cells,colors=[color])
    label = plt.label([name], x=[30],y=[cells[30]+100], colors=[color])

plt.show()

In [11]:
plt.figure(title='Income in USD Cumulated')
line=plt.bar(np.arange(60),nb.sheet.rows["tft_cumul_value_usd"].cells[startmonth:60+startmonth])
plt.show()
plt.figure(title='ROI = Return On Investment')
line=plt.plot(np.arange(60),nb.sheet.rows["roi"].cells[startmonth:60+startmonth])
plt.show()
md(f"""
- The Return On investment is compared to cost to buy all the hardware.
- The costs of the rackspace and power have been paid with income of TFT, so its the net result which is shown.
- ROI is in **{nb.roi_months}** months (starting month:{startmonth}).
- the ROI after 60 months' is:  **{round(nb.roi_end,1)}**
""")


- The Return On investment is compared to cost to buy all the hardware.
- The costs of the rackspace and power have been paid with income of TFT, so its the net result which is shown.
- ROI is in **9** months (starting month:24).
- the ROI after 60 months' is:  **8.1**
